# Classification Model for Business

Dataset for the Project was downloaded from Microsoft; the 3 data sammples was merged, explored, munged and exported to 
the file **AW__dataset**

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [2]:
#Import Dataset to build model
AW_data = pd.read_csv('C:/Users/user/Downloads/AW_dataset.csv', sep=',', index_col=0)
AW_data.head(); AW_data.columns

AW_test = pd.read_csv('C:/Users/user/Downloads/AW_test.csv', sep=',')
#Feature Engineering

AW_test['CustomerAge']=(pd.to_datetime('1998-01-01') - pd.to_datetime(AW_test['BirthDate'])).astype(np.timedelta64(1,'Y'))
AW_test['CustomerAge']=AW_test['CustomerAge'].astype(np.int64)
Age_group = {'A1':'Under 25','A2':'Between 25-45', 'A3':'Between 45-55','A4':'Over 55'}
AW_test['CustomerAgegr']=['A1' if x<25 else 'A2' if (x>= 25 and x<=45) else 'A3' if (x>=45 and x<=55) else 'A4' for x in AW_test['CustomerAge']]
AW_test['CustomerAgegr']=[Age_group[x] for x in AW_test['CustomerAgegr']]

AW_dummy = AW_data.drop(['AveMonthSpend','BikeBuyer'], axis=1)

AW_tests = pd.concat([AW_dummy,AW_test], axis=0)
label = AW_data['AveMonthSpend'] #import, and feature engineering.

In [3]:
#Split Features and labels
labels = AW_data.iloc[:,-1].values

#Numeric features
Numericfeatures = ['NumberCarsOwned','NumberChildrenAtHome','YearlyIncome']


In [4]:
#Categorical features
Catfeatures = ['City','Education','Occupation','Gender','MaritalStatus','CustomerAgegr']

#Function to transfore and encode categorical features
def encodestring(cat_feature):
    from sklearn.preprocessing import OneHotEncoder, LabelEncoder
    lab = LabelEncoder()
    lab.fit(cat_feature)
    feature = lab.transform(cat_feature)
    
    enc = OneHotEncoder()
    enc.fit(feature.reshape(-1,1))
    feature = enc.transform(feature.reshape(-1,1)).toarray()
    return feature

#Encode the feature - CountryRegionName
feature = encodestring(AW_data['CountryRegionName'])

#Loop to encode other categorical features
for cat in Catfeatures:
    temp = encodestring(AW_data[cat])
    feature = np.concatenate([feature, temp], axis=1)
feature.shape

#Concatenate Categorical features with Numeric
feature =  np.concatenate([feature,np.array(AW_data[Numericfeatures])], axis =1)
print(feature)
feature.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

[[1.00000e+00 0.00000e+00 0.00000e+00 ... 0.00000e+00 0.00000e+00
  1.37947e+05]
 [1.00000e+00 0.00000e+00 0.00000e+00 ... 1.00000e+00 3.00000e+00
  1.01141e+05]
 [1.00000e+00 0.00000e+00 0.00000e+00 ... 1.00000e+00 3.00000e+00
  9.19450e+04]
 ...
 [0.00000e+00 1.00000e+00 0.00000e+00 ... 2.00000e+00 0.00000e+00
  1.33053e+05]
 [1.00000e+00 0.00000e+00 0.00000e+00 ... 2.00000e+00 0.00000e+00
  3.19300e+04]
 [0.00000e+00 0.00000e+00 0.00000e+00 ... 2.00000e+00 0.00000e+00
  5.93820e+04]]


(16404, 294)

In [5]:
#Features Encoding

AW_feature = encodestring(AW_tests['CountryRegionName'])
#Loop to encode other categorical features
for cat in Catfeatures:
    temp = encodestring(AW_tests[cat])
    AW_feature = np.concatenate([AW_feature, temp], axis=1)
#Concatenate Categorical features with Numeric
AW_feature =  np.concatenate([AW_feature,np.array(AW_tests[Numericfeatures])], axis =1)

#Scaling of Data using scale

from sklearn.preprocessing import scale as sc
AW_feature[:,-3:] = sc(AW_feature[:,-3:])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

## Model Implementation

## Feature Selection

In [6]:
from sklearn.feature_selection import VarianceThreshold, RFECV
sel = VarianceThreshold(threshold=(0.8*(1-0.8)))
featuresreduced = sel.fit_transform(AW_feature)
print(featuresreduced.shape)
print(AW_feature.shape)
#Test dataset
#AWtest = AW_feature[-500:,:]
#AWtrain = AW_feature[:-500,:]

(16904, 13)
(16904, 296)


Prediction of the Average Monthly Spending of Customer 

In [7]:
from sklearn.model_selection import KFold
fold = KFold(n_splits=10,shuffle=True)

classifier = LinearRegression()
selector = RFECV(estimator= classifier, cv=fold)
selector = selector.fit(featuresreduced[:-500],label)
selector.support_


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True])

In [8]:
selector.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1])

In [9]:
featuresreduced = selector.transform(featuresreduced)
featuresreduced.shape

(16904, 12)

In [10]:
classifier = LinearRegression(fit_intercept=True, normalize=True)
classifier.fit(featuresreduced[:-500], label)
AW_label = classifier.predict(featuresreduced[-500:])
AW_label;

## Write Data to Excel

In [12]:
import xlsxwriter
workbook = xlsxwriter.Workbook('result.xlsx')
worksheet = workbook.add_worksheet()
row=0
col=2
for label in (AW_label):
    worksheet.write(row,col,label)
    row +=1
workbook.close()